### Importing Libraries

In [1]:
!conda install beautifulsoup4

from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

Solving environment: done

# All requested packages already installed.



### Setting maxcolwidth to 800

In [2]:
pd.set_option('max_colwidth', 800)

### Assigning the source web page to the variable and initializing Beautiful Soup to the soup variable 

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

### Initializing csv_writer object and writing the names of the columns in it as the first row

In [4]:
csv_file = open('toronto_details.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

### Scraping the web page and extracting the data table

In [5]:
table = soup.find('table', class_ = 'wikitable sortable')
rows = table.find_all('tr')

postcodes = []
boroughs = []
neighbourhoods = []

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned' :
            
            Postcode = columns[0].text
            postcodes.append(Postcode)
            
            Borough = columns[1].text
            boroughs.append(Borough)
            
            Neighbourhood = columns[2].text.split('\n')[0]
            
            if Neighbourhood == 'Not assigned':
                Neighbourhood = Borough            
                
            neighbourhoods.append(Neighbourhood)
             
    except Exception as e :
        pass 
    
postcode_explored = []
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f]
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds])
        postcode_explored.append(postcode_i)
     

### Closing the csv file

In [6]:
csv_file.close()

### Creating the pandas dataframe and displaying it

In [9]:
df = pd.read_csv('toronto_details.csv')
df1 = df[df.Borough != 'Not assigned\n']
df1.set_index('Postcode', inplace = True)
df1.reset_index(inplace = True)
df1.head(12)

,Postcode,Borough,Neighbourhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"
5,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village"
6,M1B\n,Scarborough\n,"Malvern, Rouge"
7,M3B\n,North York\n,Don Mills
8,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
9,M5B\n,Downtown Toronto\n,"Garden District, Ryerson"


### Displaying the shape of the dataframe

In [11]:
df1.shape

(103, 3)